In [9]:
from sklearn.neural_network import MLPRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn import preprocessing
from scipy.stats import randint as sp_randint

In [2]:
data = np.loadtxt('../Data/features.csv',delimiter=',', skiprows=1) # skip row for column names

In [5]:
test_ind = np.array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
       34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66,
       68, 70, 72, 74, 76, 78, 80, 82, 83])

train_ind = np.array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
       35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67,
       69, 71, 73, 75, 77, 79, 81])

secondary_ind = np.array([ 84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123])

In [3]:
cycle_life = np.log10(data[:,0])
DeltaQ_logVar = data[:,1]
DeltaQ_Min = np.log10(-data[:, 2])
DeltaQ_Skew = data[:, 3]
DeltaQ_Kurt = data[:, 4]
QD_Max_2 = data[:, 5]
QD_2 = data[:, 6]
slope_capacity_fade_2_100 = data[:, 7]
intercept_capacity_fade_2_100 = data[:, 8]
slope_capacity_fade_91_100 = data[:, 9]
intercept_capacity_91_100 = data[:, 10]
init_avg_charge_time = data[:, 11]
avg_T = data[:, 12]
min_IR = data[:, 13]
IR_100_2 = data[:, 14]

In [40]:
X = np.array([DeltaQ_logVar, DeltaQ_Min, DeltaQ_Kurt, QD_Max_2,
             slope_capacity_fade_2_100,slope_capacity_fade_91_100, 
              init_avg_charge_time, min_IR, IR_100_2]).transpose()

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

x_train = X[train_ind]
y_train = cycle_life[train_ind]

x_test = X[test_ind]
y_test = cycle_life[test_ind]

x_stest = X[secondary_ind]
y_stest = cycle_life[secondary_ind]

print(np.average(X[:,0]))

1.3788253692925331e-16


In [97]:
regressor= MLPRegressor(random_state=1)

param_grid = {
    'hidden_layer_sizes': [(100,100,50), (250,250)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.005],
    'learning_rate': ['constant','adaptive'],
    'verbose': [True]
}

folds = KFold(n_splits = 4, shuffle = True, random_state = 42)
model= RandomizedSearchCV(estimator= regressor, n_iter = 100, param_distributions = param_grid ,scoring="neg_mean_squared_error",cv=folds, random_state=42,verbose=2, return_train_score=True, n_jobs = -1)
model.fit(x_train,y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits
Iteration 1, loss = 2.65064227
Iteration 2, loss = 1.60034703
Iteration 3, loss = 0.82470195
Iteration 4, loss = 0.31911262
Iteration 5, loss = 0.06387277
Iteration 6, loss = 0.01704844
Iteration 7, loss = 0.11191811
Iteration 8, loss = 0.26680089
Iteration 9, loss = 0.40745436
Iteration 10, loss = 0.48753889
Iteration 11, loss = 0.49350795
Iteration 12, loss = 0.43619576
Iteration 13, loss = 0.33894274
Iteration 14, loss = 0.22813074
Iteration 15, loss = 0.12709295
Iteration 16, loss = 0.05256690
Iteration 17, loss = 0.01291893
Iteration 18, loss = 0.00786163
Iteration 19, loss = 0.02968788
Iteration 20, loss = 0.06594449
Iteration 21, loss = 0.10301615
Iteration 22, loss = 0.12963107
Iteration 23, loss = 0.13926064
Iteration 24, loss = 0.13086738
Iteration 25, loss = 0.10811997
Iteration 26, loss = 0.07763101
Iteration 27, loss = 0.04686389
Iteration 28, loss = 0.02223073
Iteration 29, loss = 0.00773109
Iteration 30, loss

RandomizedSearchCV(cv=KFold(n_splits=4, random_state=42, shuffle=True),
                   estimator=MLPRegressor(random_state=1), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu',
                                                       'logistic'],
                                        'alpha': [0.0001, 0.001, 0.005],
                                        'hidden_layer_sizes': [(100, 100, 50),
                                                               (250, 250),
                                                               (1000,)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'solver': ['sgd', 'adam'],
                                        'verbose': [True]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_squared_error', verbos

In [98]:
print("Best RMS score: ", model.best_score_)
print("Best hyperparameters: ", model.best_params_)

Best RMS score:  -0.009895494118874255
Best hyperparameters:  {'verbose': True, 'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (1000,), 'alpha': 0.005, 'activation': 'logistic'}


In [99]:
regressor = MLPRegressor(solver=model.best_params_['solver'],
                learning_rate=model.best_params_['learning_rate'], 
                hidden_layer_sizes = model.best_params_['hidden_layer_sizes'], 
                alpha=model.best_params_['alpha'],
                activation=model.best_params_['activation'])
regressor.fit(x_train,y_train)

MLPRegressor(activation='logistic', alpha=0.005, hidden_layer_sizes=(1000,),
             learning_rate='adaptive')

In [100]:
print("RMSE: ", np.sqrt(np.average((10**regressor.predict(x_train) - 10**y_train)**2))) 

RMSE:  258.7207771339234


In [101]:
print("RMSE: ", np.sqrt(np.average((10**regressor.predict(x_test) - 10**y_test)**2))) 


RMSE:  301.09738606551156


In [102]:
print("RMSE: ", np.sqrt(np.average((10**regressor.predict(x_stest) - 10**y_stest)**2))) 


RMSE:  462.4829373979832
